# MEIRL

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# with initialization
area = "london"
filename = "data/" + area + "/adjacency_matrix.csv"
adjacency_matrix = pd.read_csv(filename, index_col=0)

In [3]:
expert_feature_count = adjacency_matrix.copy()
expert_feature_count["value"] = 0
expert_feature_count = expert_feature_count["value"]
expert_feature_count = expert_feature_count.to_frame()
learner_feature_count = expert_feature_count.copy()
test_feature_count = expert_feature_count.copy()

In [4]:
# cross_validation
all_data = 400
training_number = int(all_data * 0.75)
test_number = int(all_data * 0.25)

# Import expert trajectory

### prepare dataset for training

In [5]:
for i in range (1,training_number):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(expert_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        expert_feature_count.loc[int(j), ["value"]] = new_count
expert_feature_count = (expert_feature_count - expert_feature_count.min()) / expert_feature_count.max() - expert_feature_count.min()    

### prepare for testing

In [6]:
for i in range (training_number + 1 , all_data):
    filename = "data/" + area + "/" + str(i)
    file = open(filename, "r")
    expert_trajectory = file.read().split(', ')
    for j in expert_trajectory:
        current_count = int(test_feature_count.loc[int(j), ["value"]].values[0])
        add_count = expert_trajectory.count(j)
        new_count = current_count + add_count 
        test_feature_count.loc[int(j), ["value"]] = new_count
test_feature_count = (test_feature_count - test_feature_count.min()) / test_feature_count.max() - test_feature_count.min()    

# Let's train

In [10]:
#Initialization of reward parameter
reward_parameter = expert_feature_count.copy()
reward_parameter.columns = ["value"]

In [11]:
def update_position(current_position, eps, reward):
    action_list = np.array(adjacency_matrix[adjacency_matrix[current_position] == 1].index)
    value_list = reward[reward.index.isin(action_list)]

    if np.random.random() > eps:
        updated_position = random.choice(value_list[value_list["value"] == max(value_list["value"])].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]
    else:
        updated_position = random.choice(value_list["value"].index.tolist())
#         gained_reward = value_list.loc[updated_position, "value"]

    return updated_position

In [13]:
# how many iterations?
Y = []
iteration = 50
for i in range (iteration):
    epsilon = 1 / (i+1)  
    learner_feature_count["value"] = 0
    for j in range (1,training_number):
        start_position = random.choice(adjacency_matrix.columns)
        current_position = start_position
        # agent will walk for 50 steps
        for k in range(100):
            current_position = str(update_position(current_position, epsilon, reward_parameter))
            learner_feature_count.loc[int(current_position), ["value"]] += 1
    
    learner_feature_count = (learner_feature_count - learner_feature_count.min()) / (learner_feature_count.max() - learner_feature_count.min() )   
    gradient = (expert_feature_count - learner_feature_count) 
    reward_parameter = (gradient*0.2) + reward_parameter
    reward_parameter = (reward_parameter - reward_parameter.min()) / (reward_parameter.max() - reward_parameter.min() )   
    
    difference = gradient.abs().sum().values[0]
    Y.append(difference)
    print(i, difference)
    
    correlation_dataframe = pd.concat([learner_feature_count, test_feature_count], axis=1)
    correlation = correlation_dataframe.corr().values[0][1]
    print(correlation)

0 219.395384615
0.234012293507
1 84.9241258741
0.414713970898
2 97.4623137221
0.309452555938
3 98.2487088883
0.425751133742
4 99.3531185031
0.35359561549
5 99.7224619056
0.371054930197
6 100.365587329
0.341705420766
7 100.809116809
0.432999113658
8 99.8255099533
0.353753436832
9 104.164842681
0.282252074445
10 105.646208726
0.338537214468
11 103.199230769
0.303008650909
12 101.597677075
0.297987044997
13 102.259917921
0.428930280267
14 106.215576147
0.358111466395
15 103.330411102
0.321375119907
16 102.793383232
0.390732004485
17 102.906898392
0.323053183614
18 103.224167623
0.364354138279
19 106.364669527
0.320276859119
20 107.806162587
0.348636196531
21 105.326731249
0.326232783731
22 103.137497175
0.314693870407
23 103.572165323
0.355291222011
24 101.216971774
0.338349100149
25 101.93510809
0.355908050486
26 103.603264033
0.345449559828
27 103.15
0.350427172702
28 106.785066656
0.277962423375
29 105.592870544
0.33979673222
30 108.348662207
0.353638435054
31 107.046596859
0.312380617

# Local Depth

In [11]:
#local depth
filename = "data/" + area + "/" + "closeness_centrality.csv"

closeness_centrality = pd.read_csv(filename, index_col=0)
adjacent_node = adjacency_matrix.sum().to_frame()
adjacent_node.columns = ["closeness_centrality"]
adjacent_node.index = adjacent_node.index.map(int)
mean_depth = (closeness_centrality - adjacent_node) / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   

In [12]:
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_local_depth = - correlation_dataframe.corr().values[0][1]

In [13]:
correlation_local_depth

0.45406804231592879

# Connectivity 

In [14]:
mean_depth = adjacent_node / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_connectivity = correlation_dataframe.corr().values[0][1]
correlation_connectivity

0.32460389310488996

# Global Depth

In [15]:
mean_depth = closeness_centrality / (adjacent_node.shape[0] - 1)
relative_asymmetry = (2*(mean_depth - 1)) / (adjacent_node.shape[0] - 2)
relative_asymmetry = (relative_asymmetry - relative_asymmetry.min()) / (relative_asymmetry.max() - relative_asymmetry.min() )   
correlation_dataframe = pd.concat([relative_asymmetry, test_feature_count], axis=1)
correlation_global_depth = - correlation_dataframe.corr().values[0][1]
correlation_global_depth

0.45406704994670616